## RouterAgent

智能体路由可以扩展为更广泛的场景，比如根据不同的工具集合、向量库类型等，配置为不同的智能体，
然后根据用户的意图和场景，使用 RouterAgent 动态选择最佳的智能体。

RouterAgent 有两个参数：
- condition 选择函数，可以是普通函数、 lambda 函数，或者实现了 `__call__` 方法的对象
    - condition 接受两个参数，一是提示语，二是智能体名称列表
- agents 智能体列表

In [1]:
from illufly.types import RouterAgent
from illufly.chat import ChatQwen, ChatZhipu, FakeLLM

agent = RouterAgent(
    (lambda prompt, names: "Zhipu"),
    [FakeLLM(), ChatQwen(), ChatZhipu()]
)

agent("你是什么模型？")

我是一个基于人工智能的语言模型，具体来说是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型 GLM-4。我的任务是针对用户的问题和要求提供适当的答复和支持。


'我是一个基于人工智能的语言模型，具体来说是基于清华大学 KEG 实验室和智谱 AI 公司于 2024 年共同训练的语言模型 GLM-4。我的任务是针对用户的问题和要求提供适当的答复和支持。'